# Bicycle Crash analysis NEISS data 1999 to 2018
## National Electronic Injury Surveillance System

https://github.com/mrcorbett/MET_CS677_DataScienceWithPython/tree/master


"CPSC’s National Electronic Injury Surveillance System (NEISS) is a national probability sample of hospitals in the U.S. and its territories. Patient information is collected from each NEISS hospital for every emergency visit involving an injury associated with consumer products."

https://catalog.data.gov/dataset/cpscs-national-electronic-injury-surveillance-system-neiss
https://www.cpsc.gov/cgibin/NEISSQuery/home.aspx



In [1]:
# Imports

import calendar
from code_id_translator import *
from datetime import datetime
import graphviz
from IPython.display import display
import ipywidgets as widgets
from ipywidgets import interact, fixed
import matplotlib.dates as mdates
from matplotlib.lines import Line2D
import matplotlib.pyplot as plt
from neiss_backend import *
import numpy as np
import os
import pandas as pd
import pickle
import re
from scipy.stats import chi2_contingency
from scipy.stats import chi2
from sklearn import metrics
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_graphviz, plot_tree
import seaborn as sns
import sys
from time_based_graphs import *
import xlrd



In [2]:
%matplotlib inline
plt.style.use('seaborn-whitegrid')

In [3]:
class Includes():
    TimeBasedGraphs = False
    PearsonChiSquared = True
    PythonCorr = True
    GaussianNB = True
    LinearRegressionChi2 = True


- Selection of geographic areas called primary sampling units (PSU) that are defined within sampling strata. 

https://www.cdc.gov/nchs/nhis/singleton_psu.htm

## Read the pickled neissYYYY.xlsx file

xlsx_to_pckl.ipynb is used to create the pickled file

In [4]:
neiss_pathname = os.getcwd() + '/data/NEISS'

pckl_fname = neiss_pathname + '/neiss_data.pckl'
if os.path.exists(pckl_fname):
    print("Reading {}  ... ".format(pckl_fname), end="")
    dfNeiss = pickle.load( open( pckl_fname, "rb" ) )
    print("done!")
else:
    raise Exception('ERROR:  {} does not exist'.format(pckl_fname))


Reading /Users/mcorbett/Boston University/MET_CS677_DataScienceWithPython/Project/data/NEISS/neiss_data.pckl  ... done!


In [ ]:
dfNeiss.shape

(7352927, 19)

In [ ]:
dfNeiss.head()

,CPSC_Case_Number,Treatment_Date,Age,Sex,Race,Other_Race,Body_Part,Diagnosis,Other_Diagnosis,Disposition,Location,Fire_Involvement,Product_1,Product_2,Narrative_1,Narrative_2,Stratum,PSU,Weight
0,100001,1999-12-24,41,2,1.0,0,31,71,NaN,1,0,0,3299,0,41 YR FEMALE FELL WHILE WALKING. DX: SEVERE MU...,/RIGHT BUTTOCKS & BACK.,S,71,68.1086
1,100002,1999-12-27,80,1,2.0,0,31,57,NaN,1,0,0,611,0,80 YR MALE FELL IN BATHROOM/HE HIT RIGHT RIBS ...,DX: FRACTURED RIGHT RIBS-UPPER TRUNK,S,71,68.1086
2,100003,1999-12-27,4,1,1.0,0,75,53,NaN,1,0,0,1328,0,4 YR MALE HAD METAL LARGE WAGON WHEEL FALL & H...,DX: CONTUSIN ON HEAD/NO LOC.,S,71,68.1086
3,100005,1999-12-28,18,1,0.0,NaN,94,53,NaN,1,0,0,1205,0,CONTUSION EAR - STRUCK IN RIGHT EAR WITH BASKE...,NaN,S,7,68.1086
4,100009,1999-12-28,19,2,0.0,NaN,92,64,NaN,1,0,0,5031,0,SPRAIN THUMB - INJURED THUMB WHEN SNOWBOARDING,NaN,S,7,68.1086


### Update Neiss with column code dictionary from Excel file

In [ ]:
column_codes_fname = neiss_pathname + '/column_codes.xlsx'
column_dictionary = getColumnCodeDictionary(column_codes_fname)
Neiss.setColumnCodeDictionary(column_dictionary)

- Code to take the Neiss dictionaries for column codes and write them out to the column_codes.xlsx file.

In [ ]:
neiss = Neiss(dfNeiss)

In [ ]:
dfNeiss.shape

(7352927, 19)

## Build a categorical dataframe (with a subset of the overall data 3000 random rows)

The dataframe is built of columns that are only categorical in nature.

In [ ]:
# build the  matrix (contingency table)

categories = ['Sex', 'Race', 'Body_Part', 'Diagnosis', 'Disposition', 'Location',
    'Fire_Involvement', 'Product_1', 'Product_2', 'PSU', 'Stratum' ]

dfCategorical = dfNeiss.copy()
dfCategorical = dfCategorical.xs(categories, axis=1)
dfCategorical.dropna(inplace=True)
dfCategorical['Race'] = [int(x) for x in dfCategorical['Race']]
dfCategorical = dfCategorical.sample(3000)
dfCategorical.head()

,Sex,Race,Body_Part,Diagnosis,Disposition,Location,Fire_Involvement,Product_1,Product_2,PSU,Stratum
225679,1,1,75,59,1,1,0,649,1807,73,S
127822,2,1,82,53,1,9,0,1295,0,16,M
201132,2,1,34,57,1,1,0,1242,0,43,S
59431,2,0,31,53,1,1,0,1645,1842,7,S
171186,1,2,92,53,1,0,0,1893,0,20,C


## Get the code ID translator for the dfCategorical dataframe

In [ ]:
codeIdTranslator = CodeIdTranslatorDataFrame(dfCategorical, categories)
codeIdTranslator.transformColumns()
codeIdTranslator

                                                                  translator
Function Category                                                           
codeToId Body_Part         [{0: 1, 30: 2, 31: 3, 32: 4, 33: 5, 34: 6, 35:...
         Diagnosis         [{41: 1, 42: 2, 48: 3, 49: 4, 50: 5, 51: 6, 52...
         Disposition            [{1: 1, 2: 2, 4: 3, 5: 4, 6: 5, 8: 6, 9: 7}]
         Fire_Involvement                   [{0: 1, 1: 2, 2: 3, 3: 4, 4: 5}]
         Location          [{0: 1, 1: 2, 2: 3, 4: 4, 5: 5, 6: 6, 7: 7, 8:...
         PSU               [{1: 1, 2: 2, 3: 3, 4: 4, 5: 5, 6: 6, 7: 7, 8:...
         Product_1         [{115: 1, 126: 2, 131: 3, 134: 4, 136: 5, 138:...
         Product_2         [{0: 1, 127: 2, 140: 3, 264: 4, 276: 5, 277: 6...
         Race                   [{0: 1, 1: 2, 2: 3, 3: 4, 4: 5, 5: 6, 6: 7}]
         Sex                                                  [{1: 1, 2: 2}]
         Stratum                  [{'C': 1, 'L': 2, 'M': 3, 'S': 4, 'V': 5}]

### Show time based graphs of male/female injuries

In [ ]:
if True == Includes.TimeBasedGraphs:
    stat_name = 'Sex'
    date_name = 'Treatment_Date'

    TimeBasedGraphs(dfNeiss, Neiss.getColumnDictionary(stat_name), date_name, stat_name).show()


### Python Corr

In [ ]:
if True == Includes.PythonCorr:
    methods = {
        'pearson' : 'Pearson R Correlation (parametric)',  # not good for categorical data
            # For pearson:
            # both variables should be normally distributed
            # There should be no significant outliers
            # Each variable should be continuous
            # The two variables have a linear relationship
            # The observations are paired observations.
            # Should support homoscedascity.  Homoscedascity simply refers to ‘equal variances’.
        'kendall' : 'Kendall Tau-b rank correlation (non-parametric)',
            # The variables are measured on an ordinal or continuous scale.
            # Desirable if your data appears to follow a monotonic relationship.
        'spearman' : 'Spearman rank correlation (non-parametric)'
            # Does not assume that both datasets are normally distributed
        }


    dfCategoricalMatrices = {}

    for key in methods.keys():
        dfCategoricalMatrices[key] = dfCategorical.corr(method = key)
        
        fig, ax = plt.subplots(figsize=(10, 10))

        g = sns.heatmap(dfCategoricalMatrices[key], annot=True, linewidths=0.4, ax=ax)
        g.set_title(methods[key])

        # Fix the top and bottom margins of the heatmap
        bottom_y, top_y = plt.ylim() 
        bottom_y += 0.5 
        top_y -= 0.5 
        plt.ylim(bottom_y, top_y)

        plt.show() 


In [ ]:
if True == Includes.PythonCorr:
    for key in dfCategoricalMatrices.keys():
        print('{}:\n{}'.format(key, dfCategoricalMatrices))
        print()

In [ ]:
def getHighCorrelations(df, dfCategoricalCorrMatrix, minValue):
    high_correlations = []
    for yIndex, y in enumerate(dfCategoricalCorrMatrix.index):
        for xIndex, x in enumerate(dfCategoricalCorrMatrix.columns):
            #if xIndex >= yIndex:
            #    break

            if (x != y) and (dfCategoricalCorrMatrix[y][x] > minValue):
                Y = y
                X = x
                if len(df[X].unique()) > len(df[Y].unique()):
                    # Keep the smallest item on the X axis
                    Y, X = X, Y

                if (Y, X) not in high_correlations:
                    high_correlations.insert(-1, (Y, X))
    high_correlations.sort()
    return high_correlations

In [ ]:
if True == Includes.PythonCorr:
    high_correlations = getHighCorrelations(dfCategorical, dfCategoricalMatrices['spearman'], 0.15)
    for n in high_correlations:
        print(n)

In [ ]:
if True == Includes.PythonCorr:
    button = widgets.Button(description="Click Me!")
    output = widgets.Output()

    #print(high_correlations.values)
    select = high_correlations[0]
    #print(select)
    lCorrelations = ['{}, {}'.format(y, x) for y, x in high_correlations]
    correlationDropDownSel = widgets.Dropdown(
        options=lCorrelations,
        value=lCorrelations[0],
        description='correlations',
        disabled=False,
    )

    typeDropDownSel = widgets.Dropdown(
        options=['swarm', 'scatter'],
        value='swarm',
        description='plot_type',
        disabled=False,
    )

    wHBox = widgets.HBox([correlationDropDownSel, typeDropDownSel])
    wVBox = widgets.VBox([wHBox, button, output])

    display(wVBox)

    def on_button_clicked(b):
        with output:
            sel = correlationDropDownSel.value
            (xSel, ySel) = [x.strip() for x in sel.split(',')]
            print('-{}-, -{}-'.format(xSel, ySel))

            correlations(codeIdTranslator, xSel, ySel, typeDropDownSel.value)

            #sns.pairplot(dfSel, hue=xSel)
    button.on_click(on_button_clicked)

### PearsonChiSquared

https://machinelearningmastery.com/chi-squared-test-for-machine-learning/

'The Pearson’s chi-squared statistical hypothesis is an example of a test for independence between categorical variables.'

In [ ]:
if True == Includes.PearsonChiSquared:
    pearsonChiSquared = PearsonChiSquared(dfCategorical)
    dfCategoricalCorrMatrix = pearsonChiSquared.getCorrMatrixDataframe(categories)
    #print(dfCategoricalCorrMatrix.head())

    fig, ax = plt.subplots(figsize=(10, 10))

    #sns.heatmap(dfCorrMatrix, annot=True, mask=mask, fmt="d", linewidths=0.4, ax=ax)
    g = sns.heatmap(dfCategoricalCorrMatrix, annot=True, linewidths=0.4, ax=ax)
    g.set_title('Pearson Chi Squared')
    # Fix the top and bottom margins of the heatmap
    bottom_y, top_y = plt.ylim() 
    bottom_y += 0.5 
    top_y -= 0.5 
    plt.ylim(bottom_y, top_y)

    plt.show() 

## SKLearn - Gaussian Naive Bayes (GaussianNB)


Can perform online updates to model parameters via partial_fit method. 

For details on algorithm used to update feature means and variance online, 

see Stanford CS tech report STAN-CS-79-773 by Chan, Golub, and LeVeque:


http://i.stanford.edu/pub/cstr/reports/cs/tr/79/773/CS-TR-79-773.pdf


In [ ]:
if True == Includes.GaussianNB:
    currentState = codeIdTranslator.getState()
    codeIdTranslator.setState('id')
    dfCorrelationsId = codeIdTranslator.getDataFrame().copy()
    codeIdTranslator.setState('code')
    dfCorrelationsCode = codeIdTranslator.getDataFrame().copy()
    codeIdTranslator.setState(currentState)


In [ ]:
if True == Includes.GaussianNB:
    def buildLe(df):
        dfCorrelationsCodeLe = df.copy()
        le = {}
        for col in dfCorrelationsCodeLe.columns:
            le[col] = preprocessing.LabelEncoder()
            le[col].fit(dfCorrelationsCodeLe[col].unique())
            print('{0:12s} => {1}'.format(col, le[col].classes_))
            dfCorrelationsCodeLe[col] = le[col].transform(dfCorrelationsCodeLe[col])
        print(dfCorrelationsCodeLe)
        return le

    le = buildLe(dfCorrelationsCode.copy())

In [ ]:
if True == Includes.GaussianNB:
    outputFeature = 'Diagnosis'

    categoriesGaussianNB = categories.copy()
    categoriesGaussianNB.remove(outputFeature)

    dfGaussianNB = dfCorrelationsId.copy()
    #print(np.sort(dfGaussianNB[outputFeature].unique()))

    for category in categoriesGaussianNB:
        print(category, end=' - ')

        inputFeature = [category]

        # Train classifier
        gnb = GaussianNB()
        gnb.fit(
            dfGaussianNB[inputFeature].values,
            dfGaussianNB[outputFeature]
        )

        y_pred = gnb.predict(dfCorrelationsId[inputFeature])

        # Print results
        print("Number of mislabeled points out of a total {} points : {}, performance {:05.2f}%"
              .format(
                  dfGaussianNB.shape[0],
                  (dfGaussianNB[outputFeature] != y_pred).sum(),
                  100*(1-(dfGaussianNB[outputFeature] != y_pred).sum()/dfGaussianNB.shape[0])
        ), end=' ')

        pd.DataFrame(
            {'predicted': le[outputFeature].inverse_transform(y_pred),
             'actual':    dfGaussianNB[outputFeature]})

        #print(metrics.confusion_matrix(y_pred, dfGaussianNB[outputFeature]))
        print('complete')


In [ ]:
if True == Includes.GaussianNB:
    # Drop categories with low scores
    categoriesGaussianNB = categories.copy()
    categoriesGaussianNB.remove('Sex')
    categoriesGaussianNB.remove('Disposition')
    categoriesGaussianNB.remove('Location')
    categoriesGaussianNB.remove('Product_2')
    categoriesGaussianNB.remove('Stratum')

In [ ]:
if True == Includes.GaussianNB:
    dfGaussianNB_1 = dfGaussianNB.copy()

In [ ]:
if True == Includes.GaussianNB:
    dfGaussianNB_1.drop(['Sex', 'Disposition', 'Location', 'Product_2', 'Stratum'], axis=1, inplace=True)

In [ ]:
if True == Includes.GaussianNB:
    print(dfGaussianNB_1.shape)

    print('InputFeatures=', categoriesGaussianNB)

    for category in categoriesGaussianNB:
        print("----------------------------")
        print(category)
        print()

        inputFeature = [category]

        # Train classifier
        gnb = GaussianNB()
        gnb.fit(
            dfGaussianNB_1[inputFeature].values,
            dfGaussianNB_1[outputFeature]
        )

        y_pred = gnb.predict(dfCorrelationsId[inputFeature])
        print(y_pred)

        # Print results
        print(dfGaussianNB_1.shape)
        print("Number of mislabeled points out of a total {} points : {}, performance {:05.2f}%"
              .format(
                  dfGaussianNB_1.shape[0],
                  (dfGaussianNB_1[outputFeature] != y_pred).sum(),
                  100*(1-(dfGaussianNB_1[outputFeature] != y_pred).sum()/dfGaussianNB_1.shape[0])
        ))
        pd.DataFrame(
            {'predicted': le[outputFeature].inverse_transform(y_pred),
             'actual':    dfGaussianNB_1[outputFeature]})

        # dfGaussianNB_1[outputFeature] has 29 unique values.  That is why there are 29 columns
        print(metrics.confusion_matrix(y_pred, dfGaussianNB_1[outputFeature]))


## Linear regression (using chi2)

https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.SelectKBest.html

- f_classif:  ANOVA F-value between label/feature for classification tasks.

- mutual_info_classif: Mutual information for a discrete target.

- chi2:  Chi-squared stats of non-negative features for classification tasks.

- f_regression:  F-value between label/feature for regression tasks.

- mutual_info_regression:  Mutual information for a continuous target.
    
- SelectPercentile:  Select features based on percentile of the highest scores.

- SelectFpr:  Select features based on a false positive rate test.

- SelectFdr:  Select features based on an estimated false discovery rate.

- SelectFwe:  Select features based on family-wise error rate.

- GenericUnivariateSelect:  Univariate feature selector with configurable mode.


In [ ]:
from sklearn import datasets
from sklearn.feature_selection import SelectKBest, f_classif, mutual_info_classif, chi2, f_regression, mutual_info_regression
from sklearn.feature_selection import SelectPercentile, SelectFpr, SelectFdr, SelectFwe, GenericUnivariateSelect
from sklearn.linear_model import LinearRegression 
from sklearn.metrics import mean_squared_error, r2_score, scorer

In [ ]:
if True == Includes.LinearRegressionChi2:
    dfKBest = df2.copy()

    y = dfKBest[outputFeature]
    selector = SelectKBest(chi2, k=3)
    selector.fit(dfKBest[categories].values, y)
    selector.get_support()

    print(categories)
    print(selector.get_support())
    selected_columns = np.asarray(categories)[selector.get_support()]
    print(selected_columns)
    print(selector.scores_)

In [ ]:
if True == Includes.LinearRegressionChi2:
    X = dfKBest[selected_columns]
    print(X.shape)

In [ ]:
def plot_scatter(X,Y,R=None):
    plt.scatter(X, Y, s=32, marker='o', facecolors='none', edgecolors='k')
    if R is not None:
        plt.scatter(X, R, color='red', linewidth=0.5)
    plt.show()    


In [ ]:
if True == Includes.LinearRegressionChi2:
    for category in X:
        print('x=', category)
        x = np.asarray(dfKBest[category]).reshape(-1, 1)
        regressor = LinearRegression(normalize=True).fit(x, y)
        y_pred    = regressor.predict(x)
        plot_scatter(x, y, y_pred)
        print("R-squared score: {:.4f}".format(r2_score(y, y_pred)))
        print()
        print()


## Decision Tree

In [ ]:
def replaceValueWithStringInColumn(column_name, replace_value, with_string):
    dfDecisionTree[column_name] = [
            with_string if str(x) == '{}'.format(replace_value) else x 
            for x in dfDecisionTree[column_name]]  # Replace zeros in col='disposition' with 'Unknown'


In [ ]:
def checkForNumericValueInColumn(columnName):
    for index, value in enumerate(dfDecisionTree[columnName]):
        if type(value) == int:
            if 'Treatment_Date' in dfDecisionTree.columns:
                print('{}: Found int={} at {} - {}'.format(
                    columnName, value, index, dfDecisionTree['Treatment_Date'].iloc[index]))
            else:
                print('{}: Found int={} at {}'.format(columnName, value, index))
                


In [ ]:
def createOnHotEncodedDataframe(df):
    '''
    Take columns that are objects and turn them into multiple one-hot columns.

    Args:
        df    (pd.DataFrame):  The dataframe to convert to a one-hot dataframe

    Returns:
        dfOneHot (pd.DataFrame):  A dataframe with the original objects replaced with one-hot versions.


    Example dataframe:

        Cost(dollars)  Item
        2              'Baseball'
        5              'Baseball Glove'
        7              'Helmet'

    Will become a dataframe similar to:

        Cost(dollars)  ('Item', 'Baseball')    ('Item', 'Baseball Glove')  ('Item', 'Helmet)
        2              1                       0                           0
        5              0                       1                           0
        7              0                       0                           1

    By making a tuple for the column header it is easy to determine the variable that the one-hot
    column represents.
    '''
    dfOneHot = df.copy()

    updateFreq = 100
    print('Dots are printed every {} translations'.format(updateFreq))

    for col in df.columns:
        if df[col].dtype == object:
            print('Transforming {:20s}'.format(col), end='\t')

            unique = pd.unique(df[col].sort_values(ascending=True))
            nUnique = len(unique)
            print('nUnique={}'.format(nUnique), end='\t')

            updateIndex = updateFreq
            updateNext = updateFreq

            for index, value in enumerate(unique):
                if index == updateIndex:
                    updateIndex += updateFreq
                    updateNext += updateFreq
                    print('.', end='')
                    
                dfOneHot[(col, value)] = (dfOneHot[col] == value).astype(int)
            dfOneHot.drop(columns=[col], inplace=True)
            print('\tcomplete')
    return dfOneHot


In [ ]:

dfNeissDecisionTree = neiss.getDataFrame().copy()

# Remove columns containing NaN or columns where the number of unique items is greater than 
toBeDropped = []
for col in dfNeissDecisionTree.columns:
    if dfNeissDecisionTree[col].isnull().values.any():
        toBeDropped.append(col)

# Also remove the case number and the narrative
toBeDropped.extend(['CPSC_Case_Number', 'Narrative_1'])

# The following dates (1999 ... 2013) contain codes that do not match the column_codes table.
dfNeissDecisionTree = dfNeissDecisionTree[
    ~dfNeissDecisionTree['Treatment_Date'].dt.year.isin(list(range(1999, 2013)))]

#toBeDropped.extend(['Treatment_Date'])
dfNeissDecisionTree.drop(toBeDropped, axis=1, inplace=True)

# Remove values from dates (2014 ... 2018) that do not have column codes for them
dfNeissDecisionTree = dfNeissDecisionTree[~dfNeissDecisionTree['Product_1'].isin([1841, 1903])]
dfNeissDecisionTree = dfNeissDecisionTree[~dfNeissDecisionTree['Product_2'].isin([1841, 1903])]

dfNeissDecisionTree['mDate'] = mdates.date2num(dfNeissDecisionTree['Treatment_Date']) 
dfNeissDecisionTree.drop(['Treatment_Date'], axis=1, inplace=True)

#dfDecisionTree = Neiss.translateCodes(dfNeissDecisionTree[0 : len(dfNeissDecisionTree) // 4])  # Use 1/4 the data
#dfDecisionTree = Neiss.translateCodes(dfNeissDecisionTree[0 : len(dfNeissDecisionTree) // 2])  # Use 1/2 the data
dfDecisionTree = Neiss.translateCodes(dfNeissDecisionTree)


replaceValueWithStringInColumn('Product_1', 0, 'Zero') # Keep the zero.
replaceValueWithStringInColumn('Product_2', 0, 'Zero') # Especially for Product_2.
replaceValueWithStringInColumn('Fire_Involvement', 4, 'InvalidCode')


for col in dfDecisionTree.columns:
    print('{:20s}\tnunique={}\tnum_nulls={}\ttype={}'.format(
        col,
        dfDecisionTree[col].nunique(),
        dfDecisionTree[col].isnull().sum(),
        dfDecisionTree[col].dtype))

print('Done')

In [ ]:
#checkForNumericValueInColumn('Disposition')
checkForNumericValueInColumn('Fire_Involvement')
checkForNumericValueInColumn('Product_1')
checkForNumericValueInColumn('Product_2')

print('Done')

In [ ]:
dfOneHot = createOnHotEncodedDataframe(dfDecisionTree)

yLabels = []
for col in dfOneHot.columns:
    if col[0] == 'Diagnosis':
        yLabels.append(col)

xLabels = [value for value in dfOneHot.columns if value not in yLabels]

x = dfOneHot[xLabels]
y = dfOneHot[yLabels]



In [ ]:
clfDecisionTree = DecisionTreeClassifier() # gini is the default criterion
#clfDecisionTree = DecisionTreeClassifier(criterion="entropy")
clfDecisionTreeFit = clfDecisionTree.fit(x, y)
print(clfDecisionTreeFit)


- ***** 
- Full data model accuracy
- ***** 

In [ ]:
y_pred = clfDecisionTreeFit.predict(x)
y_pred

# Model accuracy
metrics.accuracy_score(y, y_pred)

In [ ]:
# This code generates the following error for some reason:
#    Error: neiss_2013_2018: syntax error in line 743 near ','

#dot_data=export_graphviz(clfDecisionTreeFit, out_file=None, 
#                     feature_names=xLabels,  # inputs
#                     class_names=yLabels, # outputs 
#                     filled=True, rounded=True,   
#                     special_characters=True)  
#graph = graphviz.Source(dot_data) 
#graph.render("neiss_2013_2018")

In [ ]:
#!open neiss_2013_2018.pdf

- ***** 
- Now with 70% training set, 30% testing set
- ***** 

In [ ]:
bu_id = 7286

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=bu_id)

clfGini = DecisionTreeClassifier()
clfGiniFit = clfGini.fit(X_train, y_train)
print(clfGiniFit)

y_pred = clfGiniFit.predict(X_test)
# print(y_pred)

# Model accuracy
print(metrics.accuracy_score(y_test, y_pred))
